In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_csv("../concept_labeling_outputs/iteration3/merged_dataset_with_concepts.csv")
window_df = pd.read_csv("../concept_labeling_outputs/iteration3/merged_window_labels.csv")

In [3]:
print(raw_df.shape)
print(window_df.shape)

(5862, 10)
(100, 8)


In [4]:
raw_df["magnitude"] = np.sqrt(raw_df["x-axis"]**2 + raw_df["y-axis"]**2 + raw_df["z-axis"]**2)

raw_df["motion_intensity"] = (raw_df["magnitude"] - raw_df["magnitude"].min()) / (raw_df["magnitude"].max() - raw_df["magnitude"].min())

x_norm = np.linalg.norm(raw_df["x-axis"])
y_norm = np.linalg.norm(raw_df["y-axis"])
z_norm = np.linalg.norm(raw_df["z-axis"])

raw_df["vertical_dominance"] = abs(raw_df["z-axis"]) / (abs(raw_df["x-axis"]) + abs(raw_df["y-axis"]) + abs(raw_df["z-axis"]) + 1e-9)

raw_df["static_posture"] = (raw_df["motion_intensity"] < 0.1).astype(float)

def compute_window_features(df, start, end):
    segment = df[(df["time_s"] >= start) & (df["time_s"] <= end)]
    if len(segment) == 0:
        return pd.Series({"motion_intensity": np.nan, "vertical_dominance": np.nan, "static_posture": np.nan, "magnitude": np.nan})
    mag = np.sqrt(segment["x-axis"]**2 + segment["y-axis"]**2 + segment["z-axis"]**2)
    motion_intensity = (mag.mean() - raw_df["magnitude"].min()) / (raw_df["magnitude"].max() - raw_df["magnitude"].min())
    vert_dom = np.linalg.norm(segment["z-axis"]) / (
        np.linalg.norm(segment["x-axis"]) + np.linalg.norm(segment["y-axis"]) + np.linalg.norm(segment["z-axis"]) + 1e-9
    )
    static_posture = 1.0 if motion_intensity < 0.1 else 0.0
    magnitude_mean = mag.mean()  # Average magnitude for the window
    return pd.Series({
        "motion_intensity": motion_intensity,
        "vertical_dominance": vert_dom,
        "static_posture": static_posture,
        "magnitude": magnitude_mean
    })

window_features = window_df.apply(lambda row: compute_window_features(raw_df, row["start_time"], row["end_time"]), axis=1)
window_df = pd.concat([window_df, window_features], axis=1)

raw_df.to_csv("raw_with_features.csv", index=False)
window_df.to_csv("window_with_features.csv", index=False)

In [5]:
print(raw_df[["motion_intensity", "vertical_dominance", "static_posture", "magnitude"]].describe())
print(window_df[["motion_intensity", "vertical_dominance", "static_posture", "magnitude"]].describe())

print(raw_df.shape)
print(window_df.shape)

       motion_intensity  vertical_dominance  static_posture    magnitude
count       5862.000000         5862.000000     5862.000000  5862.000000
mean           0.378415            0.236020        0.025418    10.902923
std            0.167550            0.207899        0.157404     4.635924
min            0.000000            0.000000        0.000000     0.432593
25%            0.304291            0.077399        0.000000     8.851990
50%            0.339304            0.169182        0.000000     9.820754
75%            0.442387            0.380660        0.000000    12.672939
max            1.000000            0.916763        1.000000    28.101477
       motion_intensity  vertical_dominance  static_posture   magnitude
count        100.000000          100.000000           100.0  100.000000
mean           0.378545            0.240067             0.0   10.906503
std            0.046183            0.086871             0.0    1.277834
min            0.326870            0.036564            

In [6]:
print(raw_df.head())

   user activity        timestamp  x-axis  y-axis  z-axis      time_s  \
0     3  Walking  101790131507000    1.76    0.46   -1.61  957.780029   
1     3  Walking  101790211616000    0.57    1.50   -2.30  957.860138   
2     3  Walking  101790251594000    2.56    5.48    5.52  957.900116   
3     3  Walking  101790291480000   11.73   14.90   17.08  957.940002   
4     3  Walking  101790331550000    9.43    5.01    3.60  957.980072   

   periodicity  temporal_stability  coordination  magnitude  motion_intensity  \
0          1.0                 0.5           0.5   2.429259          0.072163   
1          1.0                 0.5           0.5   2.804443          0.085723   
2          1.0                 0.5           0.5   8.188675          0.280318   
3          1.0                 0.5           0.5  25.521154          0.906743   
4          1.0                 0.5           0.5  11.268762          0.391637   

   vertical_dominance  static_posture  
0            0.420366             

In [7]:
print(window_df.head())

   window_idx  user activity  start_time  end_time  periodicity  \
0           0     3  Walking      957.75    960.75          1.0   
1           1     3  Walking       42.00     45.00          1.0   
2           2     3  Walking      871.50    874.50          0.5   
3           3     3  Walking       63.00     66.00          1.0   
4           4     3  Jogging      117.75    120.75          1.0   

   temporal_stability  coordination  motion_intensity  vertical_dominance  \
0                 0.5           0.5          0.362061            0.221105   
1                 0.5           0.5          0.345909            0.293488   
2                 0.5           0.5          0.346314            0.181147   
3                 0.5           0.5          0.358591            0.305797   
4                 0.5           0.5          0.467009            0.262989   

   static_posture  magnitude  
0             0.0  10.450431  
1             0.0  10.003502  
2             0.0  10.014706  
3         